In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

with open('../data/shakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

0.209729 M parameters


In [2]:
input_tensor, _ = get_batch('train')
input_tensor

tensor([[46,  6,  1, 58, 46, 53, 59,  1, 39, 56, 58,  1, 57, 39, 47, 42,  1, 58,
         53,  1, 46, 39, 60, 43,  1, 39,  1, 57, 58, 59, 40, 40],
        [42, 43, 57, 47, 56, 43,  1, 53, 44,  1, 58, 46, 43,  1, 52, 53, 40, 50,
         43, 57,  8,  0,  0, 31, 21, 15, 21, 26, 21, 33, 31, 10],
        [ 1, 39,  1, 46, 53, 50, 63,  1, 51, 39, 52,  8,  0, 35, 46, 43, 56, 43,
          5, 57,  1, 30, 53, 51, 43, 53,  5, 57,  1, 51, 39, 52],
        [57, 58,  1, 52, 53, 58,  8,  0,  0, 31, 46, 43, 54, 46, 43, 56, 42, 10,
          0, 24, 43, 58,  1, 46, 47, 51,  6,  1, 51, 63,  1, 57],
        [63,  1, 39, 51,  1, 21,  1, 58, 46, 39, 58,  1, 53, 59, 56,  1, 45, 53,
         53, 42,  1, 61, 47, 50, 50,  1, 43, 44, 44, 43, 41, 58],
        [50,  1, 50, 43, 39, 60, 43,  1, 58, 46, 43,  1, 44, 53, 43,  0, 13, 52,
         42,  1, 51, 39, 49, 43,  1, 51, 63,  1, 61, 39, 56, 57],
        [47, 52, 45,  1, 58, 46, 43, 47, 56,  1, 57, 39, 60, 39, 45, 43, 52, 43,
         57, 57,  1, 39, 57, 47, 42, 

In [3]:
input_tensor = torch.randint(0, vocab_size, (10, 32)).to(device)
input_tensor

tensor([[27, 49,  0, 27, 40, 48, 27, 50, 56, 10, 29, 12, 33,  8,  3, 51, 44, 17,
         52, 10, 18, 38, 27, 48, 53,  3, 29, 27, 43, 15, 34, 52],
        [ 8, 53, 32, 54, 22, 49, 17, 47, 14,  4,  2, 10, 27, 64, 18, 15, 59, 11,
         21,  2, 47,  0,  4, 51, 63,  9, 53, 12, 19,  8, 41, 16],
        [37,  7, 28,  2, 58, 23, 62, 15, 31, 63, 58,  8, 57, 30, 23, 15, 48, 43,
         60, 54, 46, 15, 63,  8, 24, 14, 63, 13, 48, 45, 42, 38],
        [10, 23, 50, 48, 38,  0,  4, 19, 29, 12, 29, 32, 55, 23, 15, 32,  4,  4,
          5, 14, 18,  1, 39,  8, 42, 16,  3, 13, 61, 17, 23, 44],
        [23, 36,  0, 18, 59, 12, 47,  2, 46, 55, 61, 48, 63, 62, 33, 61, 19, 24,
         62, 27, 39,  8, 57, 61, 52, 12,  1, 51, 48, 28, 26, 14],
        [39, 43, 32, 21, 53, 23, 61, 30, 38, 54, 23, 17, 47, 48, 13, 36, 10, 16,
         47, 25, 27, 31,  4, 38, 54, 26, 56, 17, 19, 39, 13, 28],
        [16, 15, 42, 20, 60, 47, 35, 49, 39, 49, 21, 55, 17, 35, 38, 55, 33, 34,
         58,  2, 28, 28, 27, 52, 59, 

In [10]:
logits, _ = model(input_tensor)
logits.shape

torch.Size([10, 32, 65])

In [2]:
max_iters = 1000
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.4116, val loss 4.4022
step 100: train loss 2.6568, val loss 2.6670


KeyboardInterrupt: 

In [3]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))


And thik tridcowi,
T OLon, btK

: Pet bobe t e.
S:
O-3 my chichanss ar hthif u qor he.tbar dXlasoate awice my thanstacomzo m h
Yowh$
Mtof in h be V! whedilin,

W iree sengcin lat Het d ov te, anen k pnghirileransesel lind me l.
-nser ce hiry ptupr aiss hew y mouninis k Boupetelivd whom

P:
The teSIkleo W-ndo whd Ceiiby we ath dourive we hhire s poo mo er; te

Apk hanteruf IA s ar irist m:
ENnge inle onth,
Mf Pre?

WISo myr KINLIE!
Ktiyd&is:
Sad al the E:
Fond uin cst amaraney Iry ts I f my ce wy
